#### Import required library

- Importing all the required libraries


In [19]:
import numpy as np
import random
import math
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings("ignore", category=SyntaxWarning)


#### Loading the dataset

In [21]:
data = np.load('../data/problem_3.npz')
x_real= data['x']
y_real = data['y']
print(f"x_real shape: {x_real.shape}")
print(f"y_real shape: {y_real.shape}")

x_real shape: (3, 5000)
y_real shape: (5000,)


#### Spliting the data into two sets
- splitting data into training and testing to check later if the generated formula will also predict good fitness on unseen data


In [22]:
x_train, x_test, y_train, y_test = train_test_split(x_real.T, y_real, test_size=0.2, random_state=42)

#### Defining constants, functions and variables

In [23]:
OPERATORS = ['+', '-', '*', '/']  # Basic binary operators
FUNCTIONS = ['sin', 'cos', 'log', 'tan', 'exp', 'sqrt', 'square']        # Example unary operators
VARIABLES = [f"x{i}" for i in range(x_real.shape[0])] #used to have the variables according to the dataset
def random_constant():
    return round(random.uniform(-10, 10), 2)

print(VARIABLES)

['x0', 'x1', 'x2']


#### Generate Random formula

In [24]:
def generate_random_expr(max_depth, prob_function=0.5):

    base_expr = VARIABLES.copy()
    random.shuffle(base_expr)
    

    while len(base_expr) > 1:
        left = base_expr.pop()
        right = base_expr.pop()
        op = random.choice(OPERATORS)
        base_expr.append(f"({left} {op} {right})")
    
    formula = base_expr[0]
    
    def extend_formula(expr, depth):
        if depth == 0:
            return expr
        if random.random() < prob_function:
            func = random.choice(FUNCTIONS)
            return f"{func}({extend_formula(expr, depth - 1)})"
        else: 
            op = random.choice(OPERATORS)
            new_term = random.choice(VARIABLES + [random_constant()])
            if random.random() < 0.5:
                return f"({extend_formula(expr, depth - 1)} {op} {new_term})"
            else:
                return f"({new_term} {op} {extend_formula(expr, depth - 1)})"
    
    formula = extend_formula(formula, max_depth)
    return formula

#### Transform and evaluation of the formula

In [25]:

def safe_log(x):
    if x <= 0:
        return None 
    return np.log(x)

def safe_sqrt(x):
    if x < 0:
        return None 
    return np.sqrt(x)


def transform_formula(formula):
    
    if isinstance(formula, str):
        formula = formula.replace("log", "safe_log")  # Replace 'log' with safe_log
        formula = formula.replace("sqrt", "safe_sqrt")  # Replace 'sqrt' with safe_sqrt
        formula = formula.replace("sin", "np.sin")  
        formula = formula.replace("cos", "np.cos")  
        formula = formula.replace("tan", "safe_tan")  
        formula = formula.replace("exp", "np.exp")  
        formula = formula.replace("square", "np.square")
    return formula

    
def evaluate_expr(expr, x_values):
    variables = {f"x{i}": x for i, x in enumerate(x_values)}

    if isinstance(expr, str):
        expr = expr.replace(" ", "")

    # Try evaluating the expression with x_values
    try:
        # Use Python eval function to evaluate the formula with the safe functions
        return eval(expr, {**variables, "np": np, "safe_log": safe_log, "safe_sqrt": safe_sqrt})
    except Exception as e:
            return np.nan




#### Fitness computation (MSE)

In [26]:
def compute_fitness(expr, x_values, y_values):
    y_pred = []
    for x_values in x_train: 
        prediction = evaluate_expr(expr, x_values)
        if prediction is not None:
            y_pred.append(prediction)
        else:
            y_pred.append(float('inf'))  
    y_pred = np.array(y_pred)
    mse = np.mean((y_pred - y_values) ** 2)
    return mse 


### fitness filter

- it is used to filter the formula which will have valid fitness values

In [27]:
def filter_valid_fitness(fitness_scores, population):
    valid_population = []
    valid_fitness_scores = []
    
    for i, fitness in enumerate(fitness_scores):
        if np.isfinite(fitness):  # Check if the fitness score is a valid number
            valid_population.append(population[i])
            valid_fitness_scores.append(fitness)
    
    return valid_population, valid_fitness_scores

### Selection


##### Tournment


In [28]:
def tournament_selection(population, fitness_scores, num_parents, tournament_size=3):
   
    parents = []
    population_size = len(population)
    
    if population_size < tournament_size:
        tournament_size = population_size 

    for _ in range(num_parents):
        tournament_indices = random.sample(range(population_size), tournament_size)

        tournament_scores = [fitness_scores[i] for i in tournament_indices]

        if not tournament_scores: 
            print("Error: Tournament scores list is empty.")
            continue

        best_index = tournament_scores.index(min(tournament_scores))
        best_individual_index = tournament_indices[best_index]

        parents.append(population[best_individual_index])

    return parents


In [29]:
def run_selection(population_size, fitness_scores, population):
    valid_population, valid_fitness_scores = filter_valid_fitness(fitness_scores, population)
    
    if len(valid_population) == 0:
        print("No valid fitness scores. Exiting the selection process.")
        return None, None 
    
    num_parents = population_size // 2  
    
    if len(valid_population) < 3:
        print("Not enough individuals for tournament selection. Reducing tournament size to match population.")
        tournament_parents = tournament_selection(valid_population, valid_fitness_scores, num_parents, tournament_size=len(valid_population))
    else:
        tournament_parents = tournament_selection(valid_population, valid_fitness_scores, num_parents)
    
    return tournament_parents

#### Crossover of 2 parents

In [30]:
import ast
import random


def parse_formula_to_tree(formula):
    return ast.parse(formula, mode='eval').body

def tree_to_formula(tree):
    return ast.unparse(tree)


In [31]:
def select_random_subtree(tree):

    if isinstance(tree, ast.BinOp) or isinstance(tree, ast.UnaryOp):
        children = [tree.left, tree.right] if isinstance(tree, ast.BinOp) else [tree.operand]
        if random.random() < 0.5:
            return tree
        return select_random_subtree(random.choice(children))
    return tree 

def replace_subtree(tree, target, replacement):

    if tree == target:
        return replacement
    if isinstance(tree, ast.BinOp):
        tree.left = replace_subtree(tree.left, target, replacement)
        tree.right = replace_subtree(tree.right, target, replacement)
    elif isinstance(tree, ast.UnaryOp):
        tree.operand = replace_subtree(tree.operand, target, replacement)
    return tree

def crossover(parent1, parent2):

    tree1 = parse_formula_to_tree(parent1)
    tree2 = parse_formula_to_tree(parent2)

    subtree1 = select_random_subtree(tree1)
    subtree2 = select_random_subtree(tree2)

    offspring_tree1 = replace_subtree(tree1, subtree1, subtree2)
    offspring_tree2 = replace_subtree(tree2, subtree2, subtree1)

    offspring1 = tree_to_formula(offspring_tree1)
    offspring2 = tree_to_formula(offspring_tree2)
    
    return offspring1, offspring2


In [32]:
def crossover_population(population, num_to_select):
    offspring = []
    selected_parents = random.sample(population, num_to_select)

    if len(selected_parents) % 2 != 0:
        selected_parents = selected_parents[:-1]

    for i in range(0, len(selected_parents), 2):
        parent1 = selected_parents[i]
        parent2 = selected_parents[i + 1]
        child1, child2 = crossover(parent1, parent2)
        offspring.extend([child1, child2])
    
    return offspring


#### Mutation in the formula

In [33]:
import random
import ast

def mutate_formula(formula, mutation_rate=0.3):
    if random.random() > mutation_rate:
        return formula 

    try:
        tree = ast.parse(formula, mode='eval')
    except Exception as e:
        print(f"Error parsing formula '{formula}' to AST: {e}")
        return formula  

    mutated_tree = mutate_tree(tree, mutation_rate, VARIABLES)
    return ast.unparse(mutated_tree) 

def mutate_tree(tree, mutation_rate, variables):
    """Mutates an AST tree."""
    mutations = 0
    for node in ast.walk(tree):
        if isinstance(node, ast.Constant) and random.random() < mutation_rate:
            node.value = random.uniform(-10, 10)
            mutations += 1

        elif isinstance(node, ast.BinOp) and random.random() < mutation_rate:
            node.op = random.choice([ast.Add(), ast.Sub(), ast.Mult(), ast.Div()])
            mutations += 1

        elif isinstance(node, ast.Call) and random.random() < mutation_rate:
            if isinstance(node.func, ast.Name):
                node.func.id = random.choice(["sin", "cos", "tan", "log", "exp", "sqrt", "square"])
                mutations += 1

        elif isinstance(node, ast.Name) and random.random() < mutation_rate:
            if variables is not None:
                node.id = random.choice(variables)  
                mutations += 1

    if mutations == 0:
        node = random.choice(list(ast.walk(tree)))
        if isinstance(node, ast.Constant):
            node.value = random.uniform(-10, 10)
        elif isinstance(node, ast.BinOp):
            node.op = random.choice([ast.Add(), ast.Sub(), ast.Mult(), ast.Div()])
        elif isinstance(node, ast.Call) and isinstance(node.func, ast.Name):
            node.func.id = random.choice(["sin", "cos", "tan", "log", "exp", "sqrt", "square"])
        elif isinstance(node, ast.Name):
            node.id = random.choice(VARIABLES)

    return tree



### remove duplication

#### Local search

In [34]:
def slightly_modify_formula(formula):

    if random.random() < 0.5: 
        operators = ['+', '-', '*', '/']
        return formula.replace(random.choice(operators), random.choice(operators), 1)
    else: 
        constants = [str(c) for c in range(-10, 10)] 
        return formula.replace(random.choice(constants), str(random.randint(-10, 10)), 1)


In [35]:
def local_search(formula, fitness_function, max_steps=10):
   
    current_formula = formula
    current_fitness = fitness_function(current_formula)
    
    for _ in range(max_steps):
        neighbor = slightly_modify_formula(current_formula)

        neighbor_fitness = fitness_function(neighbor)
        

        if neighbor_fitness < current_fitness:  
            current_formula = neighbor
            current_fitness = neighbor_fitness
        else:
            break
    
    return current_formula


#### Genetic algorithm

In [40]:

num_generations = 2
population_size = 10
num_to_select = population_size//2 
mutation_rate = 0.3


population = [generate_random_expr(max_depth=4) for _ in range(population_size)]

# Print each formula
# for i, formula in enumerate(population):
#     print(f"Formula {i + 1}: {formula}")


# Track the best formula and fitness
overall_best_formula = None
overall_best_fitness = float('inf')  # We want to minimize fitness
best_fitness_scores_per_generation = []
stagnation_counter = 0

for generation in range(num_generations):
    print(f"\nGeneration {generation + 1}/{num_generations}")

    # 1. **Evaluate Fitness Scores for Current Population**
    transformed_formula = [transform_formula(formula) for formula in population]
    fitness_scores = [compute_fitness(formula, x_train, y_train) for formula in transformed_formula]
    
    # Print the fitness scores (optional, for debugging)
    # for i, (formula, score) in enumerate(zip(population, fitness_scores)):
    #     print(f"Formula {i + 1}: {formula} | Fitness = {score}")
          
    best_fitness = min(fitness_scores)
    best_index = fitness_scores.index(best_fitness)
    best_formula = population[best_index]

    if best_fitness < overall_best_fitness:
        overall_best_formula = best_formula
        overall_best_fitness = best_fitness
        stagnation_counter = 0 
    else:
        stagnation_counter += 1

    best_fitness_scores_per_generation.append((generation + 1, best_fitness, best_formula))
    print(f"Best Formula in Generation {generation + 1}: {best_formula} | Fitness = {best_fitness}")


    # 2. **Select Tournament Parents**
    tournament_parents = run_selection(population_size, fitness_scores, population)
    tournament_fitness_scores = [compute_fitness(transform_formula(formula), x_train, y_train) for formula in tournament_parents]
    
    # for i, (formula, score) in enumerate(zip(tournament_parents, tournament_fitness_scores)):
    #     print(f"Tournment Formula {i + 1}: {formula} | Fitness = {score}")

    # 3. **Crossover to Generate Offspring**
    offspring = crossover_population(tournament_parents, int(len(tournament_parents)))
    #fitness of offsprings
    offspring_fitness_scores = [compute_fitness(transform_formula(formula), x_train, y_train) for formula in offspring]
    # Print offspring after crossover (optional)
    # print("\nOffspring after crossover:")
    # for i, (formula, score) in enumerate(zip(offspring, offspring_fitness_scores)):
    #     print(f"offspring Formula {i + 1}: {formula} | Fitness = {score}")

    # 4. **Mutation on the Offspring**
    mutated_offspring = [mutate_formula(child, mutation_rate) for child in offspring]

    mutate_fitness_scores = [compute_fitness(transform_formula(formula), x_train, y_train) for formula in mutated_offspring]
    
    # 5. **Apply Local Search to Mutated Offspring**
    refined_offspring = [
        local_search(child, lambda f: compute_fitness(transform_formula(f), x_train, y_train))
        for child in mutated_offspring
    ]
    
    # print("\nFitness scores for mutated offspring:")
    # for i, (formula, score) in enumerate(zip(mutated_offspring, mutated_fitness_scores)):
    #     print(f"Mutated Child {i}: Fitness = {score}")

    # 6. **Combine Parents and Offspring for Selection**
    combined_population = tournament_parents + refined_offspring
    combined_fitness_scores = [
        compute_fitness(transform_formula(formula), x_train, y_train)
        for formula in combined_population
        ]


    # Identify the best formula from the combined population
    combined_best_fitness = min(combined_fitness_scores)
    combined_best_index = combined_fitness_scores.index(combined_best_fitness)
    combined_best_formula = combined_population[combined_best_index]

    # Update overall best formula if needed
    if combined_best_fitness < overall_best_fitness:
        overall_best_formula = combined_best_formula
        overall_best_fitness = combined_best_fitness

    best_fitness_scores_per_generation.append((generation + 1, combined_best_fitness, combined_best_formula))
    print(f"Combined Best Formula in Generation {generation + 1}: {combined_best_formula} | Fitness = {combined_best_fitness}")
    
    # print("\nCombined population:")
    # for i, (formula, score) in enumerate(zip(combined_population, combined_fitness_scores)):
    #     print(f"combined Formula {i + 1}: {formula} | Fitness = {score}")

    assert len(combined_population) == len(combined_fitness_scores), "Population and fitness scores do not match in length!"

    

    # 7. **Check for Stagnation**
    if stagnation_counter >= 3:
        print("Fitness stagnated. Applying local search to improve diversity.")
        refined_population = [
            local_search(ind, lambda f: compute_fitness(transform_formula(f), x_train, y_train))
            for ind in random.sample(population, 10)
        ]
        combined_population[:10] = refined_population
        stagnation_counter = 0

    # 8. **Select the Best Individuals for the Next Generation**
    next_generation_parents = run_selection(population_size, combined_fitness_scores, combined_population)

    #fitness of next generation
    nextgen_fitness_scores = [compute_fitness(transform_formula(formula), x_train, y_train) for formula in next_generation_parents]
    # print("\nSelected parents for the next generation:")
    # for i, (formula, score) in enumerate(zip(next_generation_parents, nextgen_fitness_scores)):
    #     print(f"next Formula {i + 1}: {formula} | Fitness = {score}")

    population = next_generation_parents  + refined_offspring[:population_size - len(next_generation_parents)]
    # Print each formula
    # for i, formula in enumerate(population):
    #     print(f"Formula for new generation {i + 1}: {formula}")
print("outcomes are: ")    
print("Best formula is ", overall_best_formula)
print("Best fitness value is ", overall_best_fitness)


Generation 1/2
Best Formula in Generation 1: exp(cos(sin(tan(((x2 * x0) + x1))))) | Fitness = nan
Combined Best Formula in Generation 1: cos(4.44 / (3.11 / ((x0 + x2) / x1) - 6.03)) | Fitness = 2949.9674644305787

Generation 2/2
Best Formula in Generation 2: cos(4.44 / (3.11 / ((x0 + x2) / x1) - 6.03)) | Fitness = 2949.9674644305787
Combined Best Formula in Generation 2: cos(0.48439186164496917 / (-5.877217319183503 / ((x0 + x2) / x1) + 6.03)) | Fitness = 2935.124362487955
outcomes are: 
Best formula is  cos(0.48439186164496917 / (-5.877217319183503 / ((x0 + x2) / x1) + 6.03))
Best fitness value is  2935.124362487955
